In [1]:
import pandas as pd

### Handling multi-aircraft events

Most events involve only a single aircraft, and a minority involve multiple. For events with multiple aircraft, some of the files have a single row for the event but some have a single row for each aircraft. We are creating a unified dataset with one row for each aircraft by adding a column `event_key` which has both the event identifier `ev_id` and aircraft identifier `Aircraft_Key`

# "Engines" Data

Over 90% of aircraft have a single engine, we drop the other aircraft from our data.

**Note (C.J.)**: Should we just drop the column instead? Is there a particular bias we're worried about here?

In [ ]:
data = pd.read_csv('../data/ntsb/ntsb_engines.csv',usecols=['ev_id',
                                                            'Aircraft_Key',
                                                            'eng_type',
                                                            'eng_no'])
data['event_key'] = data['ev_id'].astype(str) + '_' + data['Aircraft_Key'].astype(str)

data = data[data['eng_no']==1].drop(columns='eng_no')
engine_dummies = pd.get_dummies(data['eng_type'],dtype=int,prefix='engine')

engines = pd.concat([data[['event_key','ev_id','Aircraft_Key']],engine_dummies],axis=1)

#export.to_csv('../data/ntsb/cleaned/engine_data.csv')

# "Aircraft" Data

In [ ]:
data = pd.read_csv('../data/ntsb/ntsb_aircraft.csv',usecols=['ev_id',
                                                             'Aircraft_Key',
                                                             'far_part',
                                                             'damage',
                                                             'acft_fire',
                                                             'acft_expl',
                                                             'acft_make',
                                                             'acft_category',
                                                             'homebuilt',
                                                             'total_seats',
                                                             'num_eng',
                                                             'fixed_retractable',
                                                             'date_last_insp',
                                                             'owner_acft',
                                                             'certs_held',
                                                             'oprtng_cert',
                                                             'oper_cert',
                                                             'type_fly',
                                                             'second_pilot',
                                                             'evacuation',
                                                             'rwy_len',
                                                             'rwy_width',
                                                             'acft_year',
                                                             'fuel_on_board',
                                                             'unmanned'])


data['event_key'] = data['ev_id'].astype(str) + '_' + data['Aircraft_Key'].astype(str)
aircraft = data.copy()

/var/folders/c8/jr_n0v1x3n52_hpzs9nlqykw0000gn/T/ipykernel_18600/3535822329.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/ntsb/ntsb_aircraft.csv',usecols=['ev_id',


# "Findings" Data

# "Event" Data

In [123]:
data = pd.read_csv('../data/ntsb/ntsb_events.csv',usecols=['ev_id',
                                                            'ev_type',
                                                            'ev_highest_injury',
                                                            'inj_f_grnd',
                                                            'inj_m_grnd',
                                                            'inj_s_grnd',
                                                            'inj_tot_f',
                                                            'inj_tot_m',
                                                            'inj_tot_m',
                                                            'inj_tot_n',
                                                            'inj_tot_s',
                                                            'inj_tot_t',
                                                            'ev_time',
                                                            'ev_year',
                                                            'ev_month',
                                                            'on_ground_collision',
                                                            'latitude',
                                                            'longitude',
                                                            'apt_dist',
                                                            'light_cond',
                                                            'wx_dew_pt',
                                                            'wind_vel_kts',
                                                            'gust_kts',
                                                            'altimeter',
                                                            ])

/var/folders/c8/jr_n0v1x3n52_hpzs9nlqykw0000gn/T/ipykernel_18600/722889367.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/ntsb/ntsb_events.csv',usecols=['ev_id',


# Joining Event Data with Aircraft-Specific Data
To join the primary "event" dataset, which is event-specific with the other tables which are aircraft-specific, we need to take a careful approach to how we go about joining them.  The general idea goes as follows:
1. Create the "event_key" variable in the aircraft-specific datasets, which takes the format '{ev_id}_{Aircraft_Key}'.
2. Join the lower-level datasets together to maximize the number of observations.  Some will likely be in one and not another, but what is important is that we collect a list of all individual aircraft-level observations.
3. Once all lower-level datasets have been joined together and we have a list of all events with multiple aircraft, we can export a "aircraft_count" variable which expresses the number of "Aircraft_Key" for every "ev_id."  
4. Join this "aircraft_count" column into the "Event" dataset - now we have a count of how many planes were involved in each event.
5. Create a function which duplicates every row in "Events" (aircraft_count - 1 times).  Thus, if there's 3 planes, we'll get 2 new rows of the event.
6. Re-create the "Aircraft" variable with a groupby() and cum_count() function, so that every row per ev_id is added to until there are no more observations left (will be clearer in the code).
7. Now that we have the dataset formatted to resemble the individual-aircraft-level data from other tables, we can create the "event_key" - our master joining variable - in the events data.
8. Join all datasets on the Events data by "event_key", "Aircraft_Key", and "ev_id" to ensure we are joining the right aircraft/event combos onto the event data.

### Question (C.J.) 
Are the code blocks below all necessary to get the dataset right? It looks like some things are being repeated, wasn't sure if that's because I'm not seeing the need or because old code hasn't been cleaned out

In [124]:
tables = pd.merge(engines,aircraft,on=['event_key','ev_id','Aircraft_Key'],how='left') # Join on all 3 to avoid _x and _y duplicate columns.  Also ensures correct specification.
tables['Aircraft_ID'] = tables.groupby('ev_id').cumcount() + 1 # Some events start at 2 - this line creates a new Aircraft Key that is uniform across all coding schemes.
tables['event_key'] = tables['ev_id'].astype(str) + '_' + tables['Aircraft_ID'].astype(str) # Resets the "event_key" variable to match our adjusted aircraft


In [125]:
aircraft_counts = pd.DataFrame(tables.groupby('ev_id')['Aircraft_ID'].count()).reset_index() # Counts how many unique values of "Aircraft_ID" per event
aircraft_counts.rename(columns={'Aircraft_ID':'aircraft_count'},inplace=True) 
aircraft_counts['aircraft_count'] = aircraft_counts['aircraft_count'].fillna(1) # Fill in missing - if no aircraft info, assume 1 

In [126]:
df = pd.merge(data,aircraft_counts,on='ev_id',how='left') # Concatenates "aircraft count" var to dataset, will indicate how many replicate rows to generate
df['aircraft_count'] = df['aircraft_count'].fillna(1)

In [127]:
df_repeated = df.loc[df.index.repeat(df['aircraft_count'])].copy() # Creates repeated rows based on # of aircraft (indicated in column we created)


In [128]:
df_repeated['Aircraft_ID'] = df_repeated.groupby('ev_id').cumcount() + 1 # Re-creates
df_repeated['event_key'] = df_repeated['ev_id'].astype(str) + '_' + df_repeated['Aircraft_ID'].astype(str)


In [129]:
merged = df_repeated.merge(tables, on=['event_key','ev_id','Aircraft_ID'], how='left')

In [130]:
merged

,ev_id,ev_type,ev_time,ev_year,ev_month,on_ground_collision,latitude,longitude,apt_dist,light_cond,...,oprtng_cert,oper_cert,type_fly,second_pilot,evacuation,rwy_len,rwy_width,acft_year,fuel_on_board,unmanned
0,20080211X00175,ACC,1907.0,2008,1,NaN,049130N,0122412W,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20080107X00026,ACC,2230.0,2008,1,N,381326N,1222659W,0.0,DAYL,...,NaN,NaN,PERS,N,NaN,2700.0,45.0,NaN,NaN,0.0
2,20080107X00026,ACC,2230.0,2008,1,N,381326N,1222659W,0.0,DAYL,...,NaN,NaN,PERS,N,NaN,2700.0,45.0,NaN,NaN,0.0
3,20080109X00036,ACC,2200.0,2008,1,NaN,293022N,0952836W,0.0,DAYL,...,NaN,NaN,PERS,N,NaN,5003.0,100.0,NaN,NaN,0.0
4,20080107X00027,ACC,825.0,2008,1,NaN,035321N,0973829W,0.0,NITE,...,NaN,NaN,PERS,N,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29067,20250425200067,ACC,2022.0,2025,4,NaN,385629N,0081574W,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29068,20250426200068,ACC,1440.0,2025,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29069,20250429200075,ACC,1420.0,2025,2,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29070,20250429200077,ACC,1305.0,2025,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Considerations Going Forward
- Do we drop observations where Aircraft_Key (from original tables) is missing? If all aircraft info is missing, in order words, do we just drop the event?